In [1]:
# -*- coding: utf-8 -*-
#!/usr/bin/env python3

import requests
import datetime
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import codecs
import time

# method #
第二題的要實作`wordcount`的功能。
考慮到可能會有很多使用者的情型，
list慢慢找可能會有些耗時，
並且排序的時間可能也會長。

所以嘗試使用`hash table` 搭配 `binary search tree`的概念…


In [40]:
# binary search tree
count = 1
class Node:
    """
    Tree node: left and right child + data which can be any object
    """
    def __init__(self, data, name=None, parent=None):
        """
        Node constructor

        @param data node data object
        """
        self.left = None
        self.right = None
        
        self.data = data
        self.name = [name]
        self.parent = parent
    def insert(self, data, name=None, parent=None):
        """
        Insert new node with data

        @param data node data object to insert
        """

        if self.data:
            if data < self.data:
                if self.left is None:
                    ret = self.left = Node(data, name, self)
                else:
                    ret = self.left.insert(data, name, self)
            elif data > self.data:
                if self.right is None:
                    ret = self.right = Node(data, name, self)
                else:
                    ret = self.right.insert(data, name, self)
            elif data == self.data:
                self.name.append(name)
                return self    
        else:
            self.name = [name]
            self.data = data
            self.parent = parent
            return self
        return ret
    def lookup(self, data, parent=None):
        """
        Lookup node containing data

        @param data node data object to look up
        @param parent node's parent
        @returns node and node's parent if found or None, None
        """
        if data < self.data:
            if self.left is None:
                return None, None
            return self.left.lookup(data, self)
        elif data > self.data:
            if self.right is None:
                return None, None
            return self.right.lookup(data, self)
        else:
            return self, parent
    def delete(self, data):
        """
        Delete node containing data

        @param data node's content to delete
        """
        # get node containing data
        node, parent = self.lookup(data)
        if node is not None:
            children_count = node.children_count()
            if children_count == 0:
                # if node has no children, just remove it
                if parent:
                    if parent.left is node:
                        parent.left = None
                    else:
                        parent.right = None
                    del node
                else:
                    self.data = None
            elif children_count == 1:
                # if node has 1 child
                # replace node with its child
                if node.left:
                    n = node.left
                else:
                    n = node.right
                if parent:
                    if parent.left is node:
                        parent.left = n
                    else:
                        parent.right = n
                    del node
                else:
                    self.left = n.left
                    self.right = n.right
                    self.data = n.data
            else:
                # if node has 2 children
                # find its successor
                parent = node
                successor = node.right
                while successor.left:
                    parent = successor
                    successor = successor.left
                # replace node data by its successor data
                node.data = successor.data
                # fix successor's parent's child
                if parent.left == successor:
                    parent.left = successor.right
                else:
                    parent.right = successor.right
    def children_count(self):
        """
        Returns the number of children

        @returns number of children: 0, 1, 2
        """
        cnt = 0
        if self.left:
            cnt += 1
        if self.right:
            cnt += 1
        return cnt
    def print_tree(self):
        """
        Print tree content inorder
        """
        if self.left:
            self.left.print_tree()
        print(self.data,  ".... fuck")
        if self.right:
            self.right.print_tree()
    def print_node(self):
        print(self.data)
    def write_file(self, file):
        global count
        if self.left:
            self.left.write_file(file)
        if self.name:
            for name in self.name:
                if count > 10:
                    break
                file.write("like #%d: %s %d\n" % (count, self.name, self.data))
                count = count + 1
        if self.right:
            self.right.write_file(file)
        

# MemberCount 類別 #

實作包含dictionary table 以及 binary search tree 的 類別。
新增的時候，直接新增至binary search tree中，並且再查詢使用者在不在tree的時候，
使用 table查找，如此可以同時解決排序問題以及找id的問題

* 時間復雜度應為O(nlogn)，其實不確定會不會比dictionary直接count，之後對最後結果排序還快，之後做實驗。
* 剛努力的想刪除節點的case，剛剛突然想到不刪其實好像沒差，效能應該差一點點而已（O(n)，常數差異可忽略）。  
 所以在 `delete_count`加了一個flag，關掉刪除node的功能。（因為目前還有bug，之後還是會實作完成）。
* 之後實作刪除功能時，root用dummy data (data=-1)，不用考慮刪到root的問題，應該會簡單一點點。


In [36]:
class MemberCount(object):
    def __init__(self):
        self.table = {}
        # data init is 1 (any id count start from 1)
        # first is a dummy node
        
        self.root = Node(0, 'root')
        self.table = {}
    def add_count_by_name(self, name):
        if name not in self.table:
            new_node = self.root.insert(1, name)
            self.table[name] = new_node
        else:
            node = self.table[name]
            count = node.data + 1
            #delete
            self.delete_count(node, name)
            #insert
            new_node = self.root.insert(count, name)
            self.table[name] = new_node
    def delete_count(self, node, name):
        # use to turn of delete node
        flag = False
        # delete name
        node.name.remove(name)
        # if name is empty, than delete this node
        # not nessasary....
        if not node.name and flag:
            print('i still in, happy')
            parent = node.parent
            if parent is not None:
                # parent's left = node's left and parent's right = node's right
                # else if parent's right not none and node is left child:
                # parent's left = node's left, node's right append to parent's right's left bottom
                # else if parent's left not none and node is right child
                # parent's right = node's right, node's left append to parent's right's left bottom 
                if parent.data > node.data:
                    parent.left = node.left
                    # travel through left
                    left_bottom = parent.right
                    while left_bottom.left is not None:
                        left_bottom = left_bottom.left
                    left_bottom.left = node.right
                elif parent.data < node.data:
                    parent.right = node.right
                    left_bottom = parent.right
                    while left_bottom.left is not None:
                        left_bottom = left_bottom.left
                    left_bottom.left = node.lift
                # hope it can work ....
                node = None
            else:
                self.root = None
        

In [41]:
start_date = 1002
end_date = 1003
like_count = MemberCount()
boo_count = MemberCount()
all_like = 0
all_boo = 0

all_file = codecs.open("all_articles.txt", "r", "utf-8")

print('search date between ' + str(start_date) + ' to ' + str(end_date))
for line in all_file:
    line_list=[x for x in line.split(',')]
    url = line_list[len(line_list)-1]
    date = int(line_list[0])

    #check date
    if end_date - start_date >= end_date - date and  end_date - date >= 0:
        print("look indside web")
        print(date)
        print(url)
        response = requests.get(url.strip())
        soup = BeautifulSoup(response.text, 'lxml')
        
        for push_soup in soup.find_all('div', 'push'):
            push_category = push_soup.find('span', 'push-tag').string.strip()
            push_name = push_soup.find('span', 'push-userid').string.strip()
#             print(push_category + ' push_id: ' + push_id)
            if push_category == '推':
                all_like = all_like + 1
#                 print('record id: ' + push_name)
                like_count.add_count_by_name(push_name)
    
push_file = codecs.open("push[%d-%d].txt"%(start_date, end_date), "w", "utf-8")
push_file.write('all like: %d' % all_like)
like_count.root.write_file(push_file)
push_file.close()
            
        
        
    

search date between 1002 to 1003
look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507043263.A.2E3.html

look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507040951.A.D41.html

look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507038027.A.08D.html

look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507037743.A.EFC.html

look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507025009.A.DC7.html

look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507022739.A.DF0.html

look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507022505.A.17B.html

look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507014555.A.A20.html

look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507008786.A.81E.html

look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507006035.A.885.html

look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507004606.A.F23.html

look indside web
1003
https://www.ptt.cc/bbs/Beauty/M.1507002711.A.E2C.html

look indside web
1003
https://www.ptt.cc/bb

In [27]:
# boo_count = MemberCount()
boo_count = Node(0, 'root')
print(boo_count.print_tree())
boo_count.parent
# print(boo_count.root.print_tree())
# print(boo_count.root.data)

None
